In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Mar  6 08:08:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import shutil
import sys

In [ ]:
train_path = "/content/drive/MyDrive/KLTN/Dataset/act/csv/train.csv"
test_path = "/content/drive/MyDrive/KLTN/Dataset/act/csv/test.csv"

In [ ]:
train_df = pd.read_csv(train_path, delimiter="\t")
test_df = pd.read_csv(test_path, delimiter="\t")

In [ ]:
train_df.head(5)

,utterance,Attraction-Inform,Attraction-NoOffer,Attraction-Recommend,Attraction-Request,Attraction-Select,Booking-Book,Booking-Inform,Booking-NoBook,Booking-Request,...,Train-NoOffer,Train-OfferBook,Train-OfferBooked,Train-Request,Train-Select,general-bye,general-greet,general-reqmore,general-thank,general-welcome
0,I'd really like to take my client out to a nic...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,I show many restaurants that serve Indian food...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,I am looking for an expensive indian restauran...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Might I recommend Saffron Brasserie? That is a...,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Sure thing, please book for 6 people at 19:30 ...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
label_list = train_df.columns[1:]
label_list

Index(['Attraction-Inform', 'Attraction-NoOffer', 'Attraction-Recommend',
       'Attraction-Request', 'Attraction-Select', 'Booking-Book',
       'Booking-Inform', 'Booking-NoBook', 'Booking-Request',
       'Hospital-Inform', 'Hospital-Request', 'Hotel-Inform', 'Hotel-NoOffer',
       'Hotel-Recommend', 'Hotel-Request', 'Hotel-Select', 'Police-Inform',
       'Police-Request', 'Restaurant-Inform', 'Restaurant-NoOffer',
       'Restaurant-Recommend', 'Restaurant-Request', 'Restaurant-Select',
       'Taxi-Inform', 'Taxi-Request', 'Train-Inform', 'Train-NoOffer',
       'Train-OfferBook', 'Train-OfferBooked', 'Train-Request', 'Train-Select',
       'general-bye', 'general-greet', 'general-reqmore', 'general-thank',
       'general-welcome'],
      dtype='object')

In [ ]:
def get_labels(df):
    labels = []
    for i in range(len(df)):
        row = []
        for j in label_list:
            if ((j in df.columns) and (df.iloc[i][j] == 1)):
                row.append(1)
            else:
                row.append(0)
        labels.append(row)
    return labels

Y_test = get_labels(test_df)

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 1e-05

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, max_len, is_test_df):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.text = self.df['utterance']
        if (is_test_df):
            self.labels = get_labels(self.df)
        else:
            self.labels = self.df[label_list].values

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'labels': torch.FloatTensor(self.labels[index])
        }

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self, num_labels):
        super(BERTClassifier, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs['pooler_output']
        pooled_output = self.dropout(pooled_output)
        logits = self.fc(pooled_output)
        return logits

In [ ]:
def train(model, training_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in training_loader:
        input_ids = batch['input_ids'].to(device, dtype = torch.long)
        attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.float)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask, token_type_ids)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(training_loader)

In [ ]:
def validate(model, validation_loader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in validation_loader:
            input_ids = batch['input_ids'].to(device, dtype = torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.float)

            logits = model(input_ids, attention_mask, token_type_ids)
            
            loss = criterion(logits, labels)
            total_loss += loss.item()
    return total_loss / len(validation_loader)

In [ ]:
def predict(model, dataloader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device, dtype = torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = batch['token_type_ids'].to(device, dtype = torch.long)

            logits = model(input_ids, attention_mask, token_type_ids)

            predictions.append(torch.sigmoid(logits).cpu().detach().numpy())
    return np.concatenate(predictions)

**Main:**

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BERTClassifier(len(label_list))
model.to(device)

In [ ]:
# Initialize optimizer and loss function
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
# Load data
train_size = 0.8
train_df2 = train_df.sample(frac=train_size, random_state=200)
val_df = train_df.drop(train_df2.index).reset_index(drop=True)
train_df=train_df2.reset_index(drop=True)

train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN, False)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=0)

valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN, False)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=0)

test_dataset = CustomDataset(test_df, tokenizer, MAX_LEN, True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
def load_checkpoint(checkpoint_path, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path)
    # Initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # Initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # Initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # Return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min

def save_checkpoint(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
# Train and validate the model
checkpoint_path = "/content/drive/MyDrive/KLTN/Model/BERT/curr_checkpoint"
best_model_path = "/content/drive/MyDrive/KLTN/Model/BERT/best_model.pt"
valid_loss_min = np.Inf

for epoch in range(EPOCHS):
    train_loss = train(model, train_dataloader, optimizer, criterion, device)
    valid_loss = validate(model, valid_dataloader, criterion, device)
    print(f'Epoch {epoch + 1}: train_loss = {train_loss:.4f}, valid_loss = {valid_loss:.4f}')

    # Create checkpoint variable and add important data
    checkpoint = {
        'epoch': epoch + 2,
        'valid_loss_min': valid_loss,
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
    }

    # Save checkpoint
    save_checkpoint(checkpoint, False, checkpoint_path, best_model_path)

    # Save the model if validation loss has decreased
    if valid_loss <= valid_loss_min:
      print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min, valid_loss))
      # Save checkpoint as best model
      save_checkpoint(checkpoint, True, checkpoint_path, best_model_path)
      valid_loss_min = valid_loss

Epoch 1: train_loss = 0.1121, valid_loss = 0.0492
Validation loss decreased (inf --> 0.049183).  Saving model ...
Epoch 2: train_loss = 0.0393, valid_loss = 0.0328
Validation loss decreased (0.049183 --> 0.032809).  Saving model ...
Epoch 3: train_loss = 0.0289, valid_loss = 0.0289
Validation loss decreased (0.032809 --> 0.028887).  Saving model ...
Epoch 4: train_loss = 0.0248, valid_loss = 0.0279
Validation loss decreased (0.028887 --> 0.027902).  Saving model ...
Epoch 5: train_loss = 0.0222, valid_loss = 0.0270
Validation loss decreased (0.027902 --> 0.027026).  Saving model ...


In [ ]:
# Test the model and evaluate
from sklearn.metrics import accuracy_score, classification_report

threshold = 0.5

y_true = Y_test
y_pred = predict(model, test_dataloader, device)

for item in y_pred:
    for j in range(len(y_pred[0])):
        if (item[j] > threshold):
            item[j] = 1
        else:
            item[j] = 0

print("Test Accuracy : {}".format(accuracy_score(y_true, y_pred)))
print("\nClassification Report : ")
print(classification_report(y_true, y_pred, target_names=label_list))

Test Accuracy : 0.7755018990775909

Classification Report : 
                      precision    recall  f1-score   support

   Attraction-Inform       0.90      0.90      0.90      1522
  Attraction-NoOffer       0.89      0.90      0.89        60
Attraction-Recommend       0.74      0.62      0.67       148
  Attraction-Request       0.87      0.62      0.73       676
   Attraction-Select       0.67      0.53      0.59        55
        Booking-Book       0.90      0.94      0.92       537
      Booking-Inform       0.92      0.90      0.91       564
      Booking-NoBook       0.98      0.93      0.95       131
     Booking-Request       0.93      0.93      0.93       321
     Hospital-Inform       0.00      0.00      0.00         0
    Hospital-Request       0.00      0.00      0.00         0
        Hotel-Inform       0.89      0.90      0.90      2156
       Hotel-NoOffer       0.83      0.88      0.86        67
     Hotel-Recommend       0.79      0.66      0.72       140
       H

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1